In [1]:
import numpy as np
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
#from scrape import simple_get
import re

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

In [3]:
def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

In [4]:
def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [21]:
state = ['AK','AL','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY',
         'LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND',
         'OH','OK','OR','PA','RI','SC','SD','TN','TX','UT', 'VT', 'VA', 'WA','WV', 'WI', 'WY']
url = 'https://www.wildflower.org/collections/printable.php?collection='

In [6]:
html = BeautifulSoup(raw_html, 'html.parser')

In [22]:
#Create an empty list to fill with native plant information.
L = []
for ST in state:
    #Get raw html for each state (ST).
    raw_html = simple_get(url+ST)
    #Parse the html.
    html  = BeautifulSoup(raw_html, 'html.parser')
    #Select the 'tr' tag (e.g., 'table row' tag).
    row = html.select('tr')
    #Get each row of commercially available native plant data by state.
    for i in range(2,len(row)):
        row1 = row[i]
        spp_sc = list(row1.children)[1].text
        spp_nm = list(row1.children)[3].text
        spp_nm = [x.strip() for x in spp_nm.split(',')][0]
        duration = list(row1.children)[5].text
        duration = [x.strip() for x in duration.split(',')][0]
        p_type = list(row1.children)[7].text
        p_type = [x.strip() for x in p_type.split(',')][0]
        sun = list(row1.children)[9].text
        sun = [x.strip() for x in sun.split(',')][0]
        water = list(row1.children)[11].text
        water = [x.strip() for x in water.split(',')][0]
        L.append([spp_sc,spp_nm,duration,p_type,sun,water,ST])
    
L = pd.DataFrame(L, columns=['Scientific_Name','Common_Name','Duration',
                             'Plant_Type','Sun_Req','Water_Req','State'])

                                 Scientific_Name                 Common_Name  \
0                               Abies lasiocarpa               Subalpine Fir   
1                                Acer circinatum           Oregon Vine Maple   
2                                   Acer glabrum        Rocky Mountain Maple   
3                           Aconitum columbianum         Columbian Monkshood   
4                               Adiantum pedatum    Northern Maidenhair Fern   
5                                Agrostis scabra              Fly-away Grass   
6                          Amelanchier alnifolia      Saskatoon Serviceberry   
7                         Anaphalis margaritacea  Western Pearly Everlasting   
8                              Anemone multifida             Pacific Anemone   
9                              Aquilegia formosa           Western Columbine   
10                       Arctostaphylos uva-ursi                Kinnikinnick   
11                             Arnica co

In [23]:
L.to_csv('plants.csv')